In [ ]:

from transformers import MBartForConditionalGeneration, MBartTokenizer, MBartConfig

from hftrim.ModelTrimmers import MBartTrimmer

import pickle 

from hftrim.TokenizerTrimmer import TokenizerTrimmer

def read_CSL_annotations(CSL_annot_path):
    
    with open(CSL_annot_path, 'rb') as f:
        data = pickle.load(f)

    return data
raw_data = read_CSL_annotations('/Users/tannicholas/CSL-Daily/sentence_label/processed/labels_train.pkl')

data = []

for video in raw_data['info']:
    sentence = video['translation']
    # gloss = value['gloss']
    data.append(sentence)
    # data.append(gloss.lower())

tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="zh_CN", tgt_lang="zh_CN")

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
model.save_pretrained('pretrain_models/mbart-large-cc25')
configuration = model.config

# trim tokenizer
tt = TokenizerTrimmer(tokenizer)
tt.make_vocab(data)
tt.make_tokenizer()

# trim model
mt = MBartTrimmer(model, configuration, tt.trimmed_tokenizer)
mt.make_weights(tt.trimmed_vocab_ids)
mt.make_model()

new_tokenizer = tt.trimmed_tokenizer
new_model = mt.trimmed_model

new_tokenizer.save_pretrained('pretrain_models/MBart_trimmed_CSL')
new_model.save_pretrained('pretrain_models/MBart_trimmed_CSL')

## mytran_model
configuration = MBartConfig.from_pretrained('pretrain_models/mytran/config.json')
configuration.vocab_size = new_model.config.vocab_size
print(configuration.vocab_size)
mytran_model = MBartForConditionalGeneration._from_config(config=configuration)
mytran_model.model.shared = new_model.model.shared

mytran_model.save_pretrained('pretrain_models/mytran_CSL')












/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}
100%|██████████| 250000/250000 [00:00<00:00, 2013027.58it/s]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/gene

6036


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}


In [168]:
mbart_trimmed = MBartForConditionalGeneration.from_pretrained('pretrain_models/mytran_CSL')
mbart_trimmed

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(6036, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(6036, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-2): 3 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fin

In [167]:
from transformers import MBartTokenizer

# Initialize the tokenizer for Chinese with source and target languages set to Chinese
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="zh_CN", tgt_lang="zh_CN")

# List of sample Chinese sentences
sentences = [
    "你好！这是一个测试。",
    "今天天气真好，我们一起去公园吧！",
    "请问最近的地铁站在哪里？",
    "我很高兴认识你。",
    "我喜欢学习新的语言。"
]

# Loop through each sentence, encode and decode
for sentence in sentences:
    # Encode the sentence
    encoded_input = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    print("Original Sentence:", sentence)
    print("Encoded Input IDs:", encoded_input['input_ids'])
    
    # Decode the encoded tokens back to text
    decoded_output = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)
    print ("")

/opt/homebrew/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Original Sentence: 你好！这是一个测试。
Encoded Input IDs: tensor([[     6, 124084,     38, 100013,  49125,     30,      2, 250025]])
Decoded Output: 你好!这是一个测试。

Original Sentence: 今天天气真好，我们一起去公园吧！
Encoded Input IDs: tensor([[ 61168,  70871,   5364,   1322,      4,   1535, 184840,  77493,   4502,
             38,      2, 250025]])
Decoded Output: 今天天气真好,我们一起去公园吧!

Original Sentence: 请问最近的地铁站在哪里？
Encoded Input IDs: tensor([[     6, 200757,  13064,     43, 112201,   5370, 164691,     32,      2,
         250025]])
Decoded Output: 请问最近的地铁站在哪里?

Original Sentence: 我很高兴认识你。
Encoded Input IDs: tensor([[ 13129, 222473,  41380,    994,     30,      2, 250025]])
Decoded Output: 我很高兴认识你。

Original Sentence: 我喜欢学习新的语言。
Encoded Input IDs: tensor([[ 13129,  14999,   7272,  12122,  38046,     30,      2, 250025]])
Decoded Output: 我喜欢学习新的语言。



In [171]:
from transformers import MBartTokenizer

# Initialize the tokenizer for Chinese with source and target languages set to Chinese
tokenizer = MBartTokenizer.from_pretrained("pretrain_models/MBart_trimmed_CSL", src_lang="zh_CN", tgt_lang="zh_CN")

# List of sample Chinese sentences
sentences = [
    "你好！这是一个测试。",
    "今天天气真好，我们一起去公园吧！",
    "请问最近的地铁站在哪里？",
    "我很高兴认识你。",
    "我喜欢学习新的语言。"
]

# Loop through each sentence, encode and decode
for sentence in sentences:
    # Encode the sentence
    encoded_input = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    print("Original Sentence:", sentence)
    print("Encoded Input IDs:", encoded_input['input_ids'])
    
    # Decode the encoded tokens back to text
    decoded_output = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)
    print ("")

Original Sentence: 你好！这是一个测试。
Encoded Input IDs: tensor([[   5, 4066,   10, 3556, 2257,    7,    2, 6034]])
Decoded Output: 你好!这是一个测试。 zh_CN

Original Sentence: 今天天气真好，我们一起去公园吧！
Encoded Input IDs: tensor([[2596, 2849,  334,   68,    4,   80, 5187, 3027,  278,   10,    2, 6034]])
Decoded Output: 今天天气真好,我们一起去公园吧! zh_CN

Original Sentence: 请问最近的地铁站在哪里？
Encoded Input IDs: tensor([[   5, 5456,  799,   11, 3839,  335, 4846,    8,    2, 6034]])
Decoded Output: 请问最近的地铁站在哪里? zh_CN

Original Sentence: 我很高兴认识你。
Encoded Input IDs: tensor([[ 803, 5672, 2014,   48,    7,    2, 6034]])
Decoded Output: 我很高兴认识你。 zh_CN

Original Sentence: 我喜欢学习新的语言。
Encoded Input IDs: tensor([[ 803,  901,  461,  750, 1900,    7,    2, 6034]])
Decoded Output: 我喜欢学习新的语言。 zh_CN



In [175]:
from transformers import MBartTokenizer

# Initialize the tokenizer for Chinese with source and target languages set to Chinese
tokenizer = MBartTokenizer.from_pretrained("pretrain_models/MBart_trimmed_CSL", src_lang="zh_CN", tgt_lang="zh_CN")

# List of sample Chinese sentences
sentences = [
    "你好！这是一个测试。",
    "今天天气真好，我们一起去公园吧！",
    "请问最近的地铁站在哪里？",
    "我很高兴认识你。",
    "我喜欢学习新的语言。"
]

# Loop through each sentence, encode and decode
for sentence in sentences:
    # Encode the sentence
    encoded_input = tokenizer(sentence, return_tensors="pt", add_special_tokens=True)
    print("Original Sentence:", sentence)
    print("Encoded Input IDs:", encoded_input['input_ids'])
    
    # Decode the encoded tokens back to text
    decoded_output = tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)
    print("Decoded Output:", decoded_output)
    print ("")

Original Sentence: 你好！这是一个测试。
Encoded Input IDs: tensor([[   5, 4066,   10, 3556, 2257,    7,    2, 6034]])
Decoded Output: 你好!这是一个测试。 zh_CN

Original Sentence: 今天天气真好，我们一起去公园吧！
Encoded Input IDs: tensor([[2596, 2849,  334,   68,    4,   80, 5187, 3027,  278,   10,    2, 6034]])
Decoded Output: 今天天气真好,我们一起去公园吧! zh_CN

Original Sentence: 请问最近的地铁站在哪里？
Encoded Input IDs: tensor([[   5, 5456,  799,   11, 3839,  335, 4846,    8,    2, 6034]])
Decoded Output: 请问最近的地铁站在哪里? zh_CN

Original Sentence: 我很高兴认识你。
Encoded Input IDs: tensor([[ 803, 5672, 2014,   48,    7,    2, 6034]])
Decoded Output: 我很高兴认识你。 zh_CN

Original Sentence: 我喜欢学习新的语言。
Encoded Input IDs: tensor([[ 803,  901,  461,  750, 1900,    7,    2, 6034]])
Decoded Output: 我喜欢学习新的语言。 zh_CN



In [151]:
data

['你们好！',
 '你们好！',
 '你们好！',
 '对不起！',
 '对不起！',
 '对不起！',
 '没关系！',
 '没关系！',
 '没关系！',
 '谢谢！',
 '谢谢！',
 '谢谢！',
 '不客气！',
 '不客气！',
 '不客气！',
 '你叫什么名字？',
 '他是谁？',
 '你叫什么名字？',
 '我叫童童。',
 '我叫童童。',
 '我叫童童。',
 '他是谁？',
 '他是谁？',
 '他是谁？',
 '他是我的手语老师。',
 '他是我的手语老师。',
 '你是老师吗？',
 '你是老师吗？',
 '我不是老师，我是学生。',
 '我不是老师，我是学生。',
 '我不是老师，我是学生。',
 '你是哪里人？',
 '你是哪里人？',
 '你是哪里人？',
 '我是上海人。',
 '我是上海人。',
 '我是上海人。',
 '他是谁？',
 '他是谁？',
 '他是谁？',
 '他是我的同学。',
 '他是我的同学。',
 '他是我的同学。',
 '他是你同学吗？',
 '他是你同学吗？',
 '他是你同学吗？',
 '他不是我的同学，他是我朋友。',
 '他不是我的同学，他是我朋友。',
 '他不是我的同学，他是我朋友。',
 '你家有几个人？',
 '你家有几个人？',
 '我家有三个人。',
 '我家有三个人。',
 '我家有三个人。',
 '你女儿几岁？',
 '你女儿几岁？',
 '你女儿几岁？',
 '李老师多大？',
 '李老师多大？',
 '李老师多大？',
 '他今年50岁。',
 '他今年50岁。',
 '他今年50岁。',
 '他女儿呢？',
 '他女儿呢？',
 '他女儿呢？',
 '他女儿今年20岁。',
 '他女儿今年20岁。',
 '他女儿今年20岁。',
 '你会打手语吗？',
 '你会打手语吗？',
 '你会打手语吗？',
 '我会打手语。',
 '我会打手语。',
 '我会打手语。',
 '你妈妈会打手语吗？',
 '你妈妈会打手语吗？',
 '你妈妈会打手语吗？',
 '他不会打手语。',
 '他不会打手语。',
 '他不会打手语。',
 '中国菜好吃吗？',
 '中国菜好吃吗？',
 '中国菜好吃吗？',
 '中国菜很好吃。',
 '中国菜很好吃。',
 '中国菜很好吃。',
 '你会做

In [19]:


# Separate each character with a space in each sentence
data_spaced = [" ".join(sentence) for sentence in data]

data_spaced
# Output: ['吃 饭 了 吗', '你 好', '今 天 是 个 好 日 子']

['你 们 好 ！',
 '你 们 好 ！',
 '你 们 好 ！',
 '对 不 起 ！',
 '对 不 起 ！',
 '对 不 起 ！',
 '没 关 系 ！',
 '没 关 系 ！',
 '没 关 系 ！',
 '谢 谢 ！',
 '谢 谢 ！',
 '谢 谢 ！',
 '不 客 气 ！',
 '不 客 气 ！',
 '不 客 气 ！',
 '你 叫 什 么 名 字 ？',
 '他 是 谁 ？',
 '你 叫 什 么 名 字 ？',
 '我 叫 童 童 。',
 '我 叫 童 童 。',
 '我 叫 童 童 。',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 我 的 手 语 老 师 。',
 '他 是 我 的 手 语 老 师 。',
 '你 是 老 师 吗 ？',
 '你 是 老 师 吗 ？',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '你 是 哪 里 人 ？',
 '你 是 哪 里 人 ？',
 '你 是 哪 里 人 ？',
 '我 是 上 海 人 。',
 '我 是 上 海 人 。',
 '我 是 上 海 人 。',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 我 的 同 学 。',
 '他 是 我 的 同 学 。',
 '他 是 我 的 同 学 。',
 '他 是 你 同 学 吗 ？',
 '他 是 你 同 学 吗 ？',
 '他 是 你 同 学 吗 ？',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '你 家 有 几 个 人 ？',
 '你 家 有 几 个 人 ？',
 '我 家 有 三 个 人 。',
 '我 家 有 三 个 人 。',
 '我 家 有 三 个 人 。',
 '你 女 儿 几 岁 ？',
 '你 女 儿 几 岁 ？',
 '你 女 儿 几 岁 ？',
 '李 老 师 多 大 ？',
 '李 老 师 多 大 ？',
 '李 老 师 多 大 ？',
 '他 今 年 5 0 岁 。',
 '他 今 

In [125]:
data_spaced

['你 们 好 ！',
 '你 们 好 ！',
 '你 们 好 ！',
 '对 不 起 ！',
 '对 不 起 ！',
 '对 不 起 ！',
 '没 关 系 ！',
 '没 关 系 ！',
 '没 关 系 ！',
 '谢 谢 ！',
 '谢 谢 ！',
 '谢 谢 ！',
 '不 客 气 ！',
 '不 客 气 ！',
 '不 客 气 ！',
 '你 叫 什 么 名 字 ？',
 '他 是 谁 ？',
 '你 叫 什 么 名 字 ？',
 '我 叫 童 童 。',
 '我 叫 童 童 。',
 '我 叫 童 童 。',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 我 的 手 语 老 师 。',
 '他 是 我 的 手 语 老 师 。',
 '你 是 老 师 吗 ？',
 '你 是 老 师 吗 ？',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '我 不 是 老 师 ， 我 是 学 生 。',
 '你 是 哪 里 人 ？',
 '你 是 哪 里 人 ？',
 '你 是 哪 里 人 ？',
 '我 是 上 海 人 。',
 '我 是 上 海 人 。',
 '我 是 上 海 人 。',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 谁 ？',
 '他 是 我 的 同 学 。',
 '他 是 我 的 同 学 。',
 '他 是 我 的 同 学 。',
 '他 是 你 同 学 吗 ？',
 '他 是 你 同 学 吗 ？',
 '他 是 你 同 学 吗 ？',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '他 不 是 我 的 同 学 ， 他 是 我 朋 友 。',
 '你 家 有 几 个 人 ？',
 '你 家 有 几 个 人 ？',
 '我 家 有 三 个 人 。',
 '我 家 有 三 个 人 。',
 '我 家 有 三 个 人 。',
 '你 女 儿 几 岁 ？',
 '你 女 儿 几 岁 ？',
 '你 女 儿 几 岁 ？',
 '李 老 师 多 大 ？',
 '李 老 师 多 大 ？',
 '李 老 师 多 大 ？',
 '他 今 年 5 0 岁 。',
 '他 今 

In [ ]:
'''Creating a char based tokenizer'''
from tokenizers import Tokenizer
from tokenizers.models import WordLevel, WordPiece
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
tokenizer = Tokenizer(WordLevel(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordLevelTrainer(special_tokens = ['<unk>', '<bos>', '<eos>'])
#sentences should be the list of all sentences in CSL-daily
tokenizer.train_from_iterator(data, trainer)
output = tokenizer.encode("他 是 我 的 手 语 老 师 ", add_special_tokens=True ).ids
print(output)#[3,7,4,1]
text = tokenizer.decode(output)
print(text)#吃 饭 了 吗

[9, 11, 6, 5, 82, 290, 58, 76]
他 是 我 的 手 语 老 师


In [158]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from tokenizers.normalizers import NFKC


tokenizer = Tokenizer(models.WordLevel(unk_token="<unk>"))




special_tokens = ["<unk>", "<pad>", "<bos>", "<eos>", "<mask>", "<si>"]
trainer = trainers.WordLevelTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.model = models.WordLevel(unk_token="<unk>")
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(data_spaced, trainer)

print(text)#吃 饭 了 吗


cls_token_id = tokenizer.token_to_id("<bos>")
sep_token_id = tokenizer.token_to_id("<eos>")
print(cls_token_id, sep_token_id)
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"<bos>:0 $A:0 <eos>:0",
    pair=f"<bos>:0 $A:0 <eos>:0 $B:1 <eos>:1",
    special_tokens=[("<bos>", cls_token_id), ("<eos>", sep_token_id)],
)
sentences = ["我们 的 校 园 又 整 洁 又 漂 亮", "吃 饭 了 吗"]

output = tokenizer.encode("我 们 的 校 园 又 整 洁 又 漂 亮",  "吃 饭 了 吗 ?", add_special_tokens=True )
print(output)#[3,7,4,1]
print(output.ids)
text = tokenizer.decode(output.ids, skip_special_tokens=False)
print(text)#吃 饭 了 吗

<bos> 我 们 的 校 园 又 整 洁 又 漂 亮 <eos> 吃 饭 了 吗 <unk> <eos>
2 3
Encoding(num_tokens=19, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[2, 9, 21, 8, 189, 822, 322, 849, 1351, 322, 415, 348, 3, 47, 110, 10, 148, 0, 3]
<bos> 我 们 的 校 园 又 整 洁 又 漂 亮 <eos> 吃 饭 了 吗 <unk> <eos>


In [160]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="right",
)

In [161]:
encoded = wrapped_tokenizer.batch_encode_plus(sentences, return_tensors="pt", padding=True)
print(encoded)

{'input_ids': tensor([[   2,    0,    8,  189,  822,  322,  849, 1351,  322,  415,  348,    3],
        [   2,   47,  110,   10,  148,    3,    1,    1,    1,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}


In [162]:
wrapped_tokenizer.batch_decode(encoded["input_ids"])

['<bos> <unk> 的 校 园 又 整 洁 又 漂 亮 <eos>',
 '<bos> 吃 饭 了 吗 <eos> <pad> <pad> <pad> <pad> <pad> <pad>']